## 2.1 Prompt Template

<div style="text-align: right"> Initial issue : 2025.04.28 </div>
<div style="text-align: right"> last update : 2025.04.28 </div>

프롬프트를 생성하는 방법 소개

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from utils import langsmith
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [3]:
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

### 1. from_template() 메서드로 프롬프트 객체 생성

In [4]:
template = "{country}의 수도는 어디인가요?"

prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [5]:
prompt.format(country="한국")

'한국의 수도는 어디인가요?'

In [6]:
type(prompt)

langchain_core.prompts.prompt.PromptTemplate

chain 만들기

In [7]:
chain = prompt | model

In [8]:
chain.invoke({"country": "한국"})

AIMessage(content='한국의 수도는 서울입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 15, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BRI3q7SdyW82oVswepNZJ2KCvErPY', 'finish_reason': 'stop', 'logprobs': None}, id='run-acd8e6bb-954a-4f97-993c-8464905020ce-0', usage_metadata={'input_tokens': 15, 'output_tokens': 8, 'total_tokens': 23, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### 2. PromptTemplate 객체 생성과 동시에 prompt 생성
- 명시적으로 input_variables를 지정

In [9]:
template = "{country}의 수도는 어디인가요?"
prompt = PromptTemplate(
    template=template,
    input_variables=["country"]
)
prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [10]:
prompt.format(country="한국")

'한국의 수도는 어디인가요?'

partial prompt 생성방법

In [11]:
# template 정의
template = "{country1}과 {country2}의 수도는 각각 어디인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["country1"],
    partial_variables={
        "country2": "미국"  # dictionary 형태로 partial_variables를 전달
    },
)

prompt

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': '미국'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [12]:
prompt.format(country1="한국")

'한국과 미국의 수도는 각각 어디인가요?'

In [13]:
prompt_partial = prompt.partial(country2="캐나다")
prompt_partial

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': '캐나다'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [14]:
prompt_partial.format(country1="대한민국")

'대한민국과 캐나다의 수도는 각각 어디인가요?'

In [15]:
chain = prompt_partial | model

In [16]:
chain.invoke("한국")

AIMessage(content='한국의 수도는 서울이고, 캐나다의 수도는 오타와입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 19, 'total_tokens': 37, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BRI3rJHxkzyvB9UVD0dfYraYWQUZC', 'finish_reason': 'stop', 'logprobs': None}, id='run-e0bbcfa9-4ead-45f6-b0a1-5f21e20700b8-0', usage_metadata={'input_tokens': 19, 'output_tokens': 18, 'total_tokens': 37, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [17]:
chain.invoke({"country1": "대한민국", "country2": "호주"})

AIMessage(content='대한민국의 수도는 서울이고, 호주의 수도는 캔버라입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 19, 'total_tokens': 38, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_129a36352a', 'id': 'chatcmpl-BRI3sVH3SIxXyr8T2fzurJRL8lrKa', 'finish_reason': 'stop', 'logprobs': None}, id='run-b10ef6c7-436b-492a-929f-31038a0a6a27-0', usage_metadata={'input_tokens': 19, 'output_tokens': 19, 'total_tokens': 38, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

partial_variables 는 날짜나 시간을 사용할 때 유용하다.

In [18]:
from datetime import datetime

# 오늘 날짜를 출력
datetime.now().strftime("%B %d")

'April 28'

In [19]:
# 날짜를 반환하는 함수 정의
def get_today():
    return datetime.now().strftime("%B %d")

In [20]:
prompt = PromptTemplate(
    template="오늘의 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n}명을 나열해 주세요. 생년월일을 표기해주세요.",
    input_variables=["n"],
    partial_variables={
        "today": get_today  # dictionary 형태로 partial_variables를 전달
    },
)

In [21]:
prompt.format(n=3)

'오늘의 날짜는 April 28 입니다. 오늘이 생일인 유명인 3명을 나열해 주세요. 생년월일을 표기해주세요.'

In [22]:
# chain 을 생성합니다.
chain = prompt | model

In [23]:
print(chain.invoke(3).content)

오늘, 4월 28일에 생일인 유명인 3명은 다음과 같습니다:

1. **우디 해럴슨 (Woody Harrelson)** - 1961년 7월 23일
2. **안젤리나 졸리 (Angelina Jolie)** - 1975년 6월 4일
3. **제임스 프랭코 (James Franco)** - 1978년 4월 19일

(참고: 위의 정보는 생일이 아닌 생년월일로 잘못 기재되었습니다. 4월 28일에 생일인 유명인으로는 다음과 같은 인물들이 있습니다.)

1. **우디 해럴슨 (Woody Harrelson)** - 1961년 7월 23일
2. **안젤리나 졸리 (Angelina Jolie)** - 1975년 6월 4일
3. **제임스 프랭코 (James Franco)** - 1978년 4월 19일

정확한 생일 정보를 제공하기 위해 추가적인 유명인을 나열하겠습니다.

1. **메이슨 딕슨 (Mason Dixon)** - 1990년 4월 28일
2. **리처드 바르톨리 (Richard Barthelmess)** - 1895년 4월 28일
3. **제이슨 리 (Jason Lee)** - 1970년 4월 28일

이 외에도 4월 28일에 태어난 유명인들이 많습니다.


In [24]:
print(chain.invoke({"today": "Jan 02", "n": 3}).content)

1. **아이작 뉴턴 (Isaac Newton)** - 1643년 1월 4일
2. **폴 리차드 (Paul Revere)** - 1735년 1월 1일
3. **J.R.R. 톨킨 (J.R.R. Tolkien)** - 1892년 1월 3일

(참고: 위의 생일은 1월 2일이 아닌 근처의 날짜로 제공되었습니다. 1월 2일에 태어난 유명인에 대한 정보는 제한적일 수 있습니다.)


### 3. 파일에서 template 읽어오기

In [25]:
from langchain_core.prompts import load_prompt

prompt = load_prompt("prompts/fruit_color.yaml", encoding="utf-8")
prompt

PromptTemplate(input_variables=['fruit'], input_types={}, partial_variables={}, template='{fruit}의 색깔이 뭐야?')

In [26]:
prompt.format(fruit="사과")

'사과의 색깔이 뭐야?'

### 4. ChatPromptTemplate  
- 대화 목록을 프롬프트로 주입할 때 유용
- 메시지는 tuple 형식으로 구성됨: (role, message)
- role의 종류는 다음이 있음
    - `"system"`: 시스템 설정 메시지. 주로 전역설정과 관련된 프롬프트.
    - `"human"` : 사용자 입력 메시지.
    - `"ai"`: AI 의 답변 메시지.

In [27]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template("{country}의 수도는 어디인가요?")
chat_prompt

ChatPromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?'), additional_kwargs={})])

In [28]:
chat_prompt.format(country="대한민국")

'Human: 대한민국의 수도는 어디인가요?'

또 다른 방법

In [29]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        # role, message
        ("system", "당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 {name} 입니다."),
        ("human", "반가워요!"),
        ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
        ("human", "{user_input}"),
    ]
)

# 챗 message 를 생성합니다.
messages = chat_template.format_messages(
    name="테디", user_input="당신의 이름은 무엇입니까?"
)
messages

[SystemMessage(content='당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 테디 입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='반가워요!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='당신의 이름은 무엇입니까?', additional_kwargs={}, response_metadata={})]

llm에 직접 주입해보기

In [30]:
model.invoke(messages)

AIMessage(content='제 이름은 테디입니다! 당신과 대화하게 되어 기쁩니다. 어떤 이야기를 나눌까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 66, 'total_tokens': 92, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0', 'id': 'chatcmpl-BRI42biuJMT6EJvBj5Bgfe33pkzTC', 'finish_reason': 'stop', 'logprobs': None}, id='run-a272a137-bb5c-4b71-ad6c-29268f3dab1c-0', usage_metadata={'input_tokens': 66, 'output_tokens': 26, 'total_tokens': 92, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

chain 만들어서 입력하기

In [31]:
chain = chat_template | model

In [32]:
chain.invoke({"name": "Teddy", "user_input": "당신의 이름은 무엇입니까?"}).content

'제 이름은 Teddy입니다! 당신과 대화하게 되어 기쁩니다. 어떤 이야기를 나눌까요?'

### 5. MessagePlaceholder  
-  프롬프트 템플릿에 어떤 역할을 사용할지 확실치 않거나, 서식 지정 중에 메시지 목록을 삽입하는 경우 유용
- 아직 확정되지 않았으나 언젠가 채워질 것을 확보할 때
- 주로 대화 내용을 기록할 때 많이 사용(chat_history)

In [33]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.",
        ),
        MessagesPlaceholder(variable_name="conversation"),
        ("human", "지금까지의 대화를 {word_count} 단어로 요약합니다."),
    ]
)
chat_prompt

ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annota

conversation 대화 목록을 나중에 추가하고자 할 때 MessagePlaceholder 사용 가능

In [34]:
formatted_chat_prompt = chat_prompt.format(
    word_count=5,
    conversation=[
        ("human", "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다."),
        ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
    ],
)

print(formatted_chat_prompt)

System: 당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.
Human: 안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다.
AI: 반가워요! 앞으로 잘 부탁 드립니다.
Human: 지금까지의 대화를 5 단어로 요약합니다.


chain으로 테스트

In [35]:
chain = chat_prompt | model | StrOutputParser()

In [36]:
# chain 실행 및 결과확인
chain.invoke(
    {
        "word_count": 5,
        "conversation": [
            (
                "human",
                "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다.",
            ),
            ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
        ],
    }
)

'테디, 새로 입사, 반갑습니다.'